**There are two major types of text summarization**:
  -	Extractive: summaries consist of text from the original document.   Extractive summaries are relatively easy to achieve. The whole concept is to find the relevant parts of the input text. 
  
  -	Abstractive: summaries can contain novel sequences of text not necessarily taken from the original text. Abstractive summaries are harder to achieve, even generating a very simple text is quite a challenge.


* This notebook is dedicated for covering different techniques of Extractive summarization.

In [1]:
import pandas as pd
import numpy as np
import textwrap
import nltk
import re
import string
import heapq
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
#from google.colab import data_table

import networkx as nx
from nltk.cluster.util import cosine_distance

#data_table.enable_dataframe_formatter()

# import pandas as pd
# import io
# import numpy as np
# import textwrap
# import nltk
# from nltk.corpus import stopwords
# from nltk import word_tokenize
# from nltk.stem import WordNetLemmatizer, PorterStemmer
# from google.colab import data_table
# data_table.enable_dataframe_formatter()

In [47]:
# from google.colab import drive
# drive.mount('drive')

In [48]:
df = pd.read_json("shuffled_hashed_rbc_dataset.json")
df = df.transpose()
df.head()
# df = df[['article_overview','article_text']]
# df.head()

,headline,article_overview,article_text,category,tags
58eda1b1ed4eb0bc52000abe8b87feb9,Песков заявил об отсутствии соглашений по укра...,Об отправке 11 вагонов с зерном для продажи со...,Песков заявил об отсутствии соглашений по укр...,politics,"[Украина, зерно, поставки, переговоры]"
db08ad6636ec7437ee065ea4feaab6b1,В Киеве попросили уточнить слова Байдена об от...,Западные страны не стали вводить превентивные ...,В Киеве попросили уточнить слова Байдена об о...,politics,"[Украина, Джо Байден, санкции, Владимир Зеленс..."
87b425dcafb5f6f99e4d812b01bc7c3e,Владелец бондов «Роснано» назвал возможный деф...,"По мнению гендиректора «Арикапитала», если гос...",Владелец бондов «Роснано» назвал возможный де...,economics,"[«Роснано», дефолт, облигации, рублевые облига..."
1483b6780bf81a8e8c7851d21bbf4071,«Тинькофф» описал схему выхода на новые рынки ...,Группа Олега Тинькова планирует выходить на за...,«Тинькофф» описал схему выхода на новые рынки...,finance,"[«Тинькофф банк», сделки M&A]"
a464c501c68db643b23b04209e980f03,"Верховный суд даст разъяснение, что считать пр...",Слишком длительные допросы и оскорбления могут...,"Верховный суд даст разъяснение, что считать п...",society,"[суд, допросы, Верховный суд]"


4. After that, we will download the needed files from NLTK.

In [49]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# **Data Exploration**:

In [50]:
df.head(3)

,headline,article_overview,article_text,category,tags
58eda1b1ed4eb0bc52000abe8b87feb9,Песков заявил об отсутствии соглашений по укра...,Об отправке 11 вагонов с зерном для продажи со...,Песков заявил об отсутствии соглашений по укр...,politics,"[Украина, зерно, поставки, переговоры]"
db08ad6636ec7437ee065ea4feaab6b1,В Киеве попросили уточнить слова Байдена об от...,Западные страны не стали вводить превентивные ...,В Киеве попросили уточнить слова Байдена об о...,politics,"[Украина, Джо Байден, санкции, Владимир Зеленс..."
87b425dcafb5f6f99e4d812b01bc7c3e,Владелец бондов «Роснано» назвал возможный деф...,"По мнению гендиректора «Арикапитала», если гос...",Владелец бондов «Роснано» назвал возможный де...,economics,"[«Роснано», дефолт, облигации, рублевые облига..."


In addition, It has been found that we have five entries with no articles. This might be due to some interraption in the internet connection while performing the webscraping.

## **Supporting Functions**

This section consists of some supporting functions that will be used frequently in this notebook.

In [2]:
def wrap(x):
    return textwrap.fill(x, replace_whitespace = False, fix_sentence_endings= True)

In [3]:
def hilight_summary(original_sentences,summary_sentences,sentences_format = 0):

    # original_article_overview = article['article_overview'].iloc[0]
    # original_sentences = article['article_text'].iloc[0]

    if(sentences_format == 0):
        # original_article_overview = " ".join([sentence for sentence in nltk.sent_tokenize(original_article_overview)])
        original_sentences = [sentence for sentence in nltk.sent_tokenize(original_sentences)]
        summary_sentences = [sentence for sentence in nltk.sent_tokenize(summary_sentences)]

    text = ''
    print("===============Summary=============================")
    for sentence in original_sentences:
        if sentence in summary_sentences:
            text += ' ' + sentence.replace(sentence, "\x1b[94m"+sentence+"\x1b[0m")
        else:
            text += ' ' + sentence

    print("\n".join(textwrap.wrap(text, width=150)))

The below code utilizes a method that was developed in the __supporting functions__ to highlight the extracted summary out of the original text.

In [53]:
df.describe()

,headline,article_overview,article_text,category,tags
count,1606,1606,1606,1606,1606
unique,1565,1569,1606,6,1542
top,Вучич объяснил отказ от санкций против России ...,Несмотря на «резкую» просьбу Шольца присоедини...,Песков заявил об отсутствии соглашений по укр...,finance,"[Банки, Ипотека]"
freq,3,3,1,339,5


To smoothen the process, it has been decided to remove these NA values.

In addition, it has been decided to perform some data cleanup before starting any summarization process, particularly cleaning up the new lines.

# **Summarization Methods**:

1.   TF-IDF for Text Summarization
2.   Luhn algorithm
3.   Cosine similarity
4.   SUMY ( A.   TextRankSummarizer , B.   LSA Summarizer )
5.   Pysummarization
6.   BERT


Other summarization techniques, that are more advanced, will be presented in a separate notebook.

## **1. TF-IDF for Text Summarization**

With this approach, we will performing the follwing:
1. Split the text into sentences.
2. Score each sentence.
3. Rank each sentence by scores.
4. Summary = top scoring sentences.


For demonstration purposes, we wil start with explaining the approach step by step. After that, we will build a function to apply the summarization on all the articles.

#### **A. Perform Summarization on One Data Sample:**

To begin with, let us use one article for summarization

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer

for headline in df.headline:
    title_of_doc = headline
    doc = df[df.headline == headline]['article_text']

Now, we will tokenize the text into sentences

In [55]:
sents = nltk.sent_tokenize(doc.iloc[0])

After that, we will create TF-IDF vectorizer object.

Note that we are using russian stopwords and L1 normlization. This is to insure that we will not be bias toward longer sentences with higher score "simply due to the fact that they contain more words".

In [56]:
featurizer = TfidfVectorizer(stop_words= stopwords.words('russian'), norm= 'l1')


Next, we call fit transform which give us back our TF-IDF matrix.

In [57]:
X = featurizer.fit_transform(sents)

The following code will show the summary along with the model score. As can be seen, it takes a single row from the TF-IDF matrix. Only non-zero values are considered once you have those values a mean function is applied.

In [58]:
def get_sentence_score(tfidf_row):
    # return the average of the non-zero values
    # of the tf-idf vector representation of a sentence
    x = tfidf_row[tfidf_row != 0]
    return x.mean()

In [59]:
#compute scores for each sentence
scores = np.zeros(len(sents))
for i in range(len(sents)):
    score = get_sentence_score(X[i,:])
    scores[i] = score

#sort the scores
sort_idx = np.argsort(-scores)

#print summary
for i in sort_idx[:5]:
    print(wrap("%.2f: %s" % (scores[i], sents[i])))

1.00: за 1 куб.
0.50: Копия есть у РБК.
0.25: стройматериалы налоги щебень Строительство
0.25: м щебня, что на 0,5% меньше, чем годом ранее.
0.25: м щебня обошелся покупателям почти в 1,5 тыс.


#### **B. Building a function to Perform Summarization on all data samples::**

- We prepared a function to summarize using tf_idf

- The first step is to tokenize the text into sentences

- Next we call fit transform which give us back our TF-IDF matrix (X)

- Finally preparing the summarization that include some extra steps such as scoring and sorting. then printing the highest scored 5 sentences

In [60]:
'''
creating the TF-IDF vectorizer object again in case we deicde to go for 
this approach directly in stead of the step by step approach. (note that we are
using russian stopwords and l1 normlization this insure that we will not be 
bias toward longer sentences which in this case would have a higher score. 
simply due to the fact that they contain more words.)
'''
featurizer = TfidfVectorizer(stop_words= stopwords.words('russian'), norm= 'l1')


#definning the scoring function again as we need it inside the summarize_TF_IDF funciton
#The function is intended to score a sentence given it's TF-IDF representation. 
def get_sentence_score(tfidf_row):
    # return the average of the non-zero values
    # of the tf-idf vector representation of a sentence
    x = tfidf_row[tfidf_row != 0]
    return x.mean()



def summarize_TF_IDF(text):
    # extract sentences
    sents = nltk.sent_tokenize(text)
    
    #perform tf-idf
    X = featurizer.fit_transform(sents)
    
    #compute scores for each sentence
    scores = np.zeros(len(sents))
    for i in range(len(sents)):
        score = get_sentence_score(X[i,:])
        scores[i] = score
    
    #sort the scores
    sort_idx = np.argsort(-scores)
    
    #return summary
    summary_text = str('')
    for i in sort_idx[:5]:
        # summary_text = summary_text + ("%.2f: %s " % (scores[i], sents[i]))
        summary_text = summary_text + ("%s " % (sents[i]))
    
    summary_text = re.sub(r'\n', ' ', summary_text)
    return(summary_text)

Now, we will apply the summarization to all the documents.

In [ ]:
tf_idf_summaries = []
for article in df['article_text']:
    tf_idf_summaries.append(summarize_TF_IDF(article))

df['TF_IDF_Summary'] = tf_idf_summaries

In [62]:
for headline in df.headline:
    title_of_doc = headline
    doc = df[df.headline == headline]

print(wrap(doc['headline'].iloc[0]))
print("========= summary ==========")
print(wrap(doc['TF_IDF_Summary'].iloc[0]))

Губернатор попросил не считать щебень полезным ископаемым
========= summary ==========
за 1 куб. Копия есть у РБК. стройматериалы налоги щебень Строительство
м щебня, что на 0,5% меньше, чем годом ранее. м щебня обошелся
покупателям почти в 1,5 тыс.


In [63]:
df[['headline', 'article_text', 'TF_IDF_Summary']]

,headline,article_text,TF_IDF_Summary
58eda1b1ed4eb0bc52000abe8b87feb9,Песков заявил об отсутствии соглашений по укра...,Песков заявил об отсутствии соглашений по укр...,"«Нет, пока точных никаких соглашений не достиг..."
db08ad6636ec7437ee065ea4feaab6b1,В Киеве попросили уточнить слова Байдена об от...,В Киеве попросили уточнить слова Байдена об о...,"Сомнений не было. «Я знал, и у нас были данные..."
87b425dcafb5f6f99e4d812b01bc7c3e,Владелец бондов «Роснано» назвал возможный деф...,Владелец бондов «Роснано» назвал возможный де...,"Она сообщила, что ведет переговоры о предостав..."
1483b6780bf81a8e8c7851d21bbf4071,«Тинькофф» описал схему выхода на новые рынки ...,«Тинькофф» описал схему выхода на новые рынки...,"Такого не будет», — сказал Хьюз. Это многолетн..."
a464c501c68db643b23b04209e980f03,"Верховный суд даст разъяснение, что считать пр...","Верховный суд даст разъяснение, что считать п...",301 УК. Об этом говорится в проекте постановле...
...,...,...,...
46414c82750c5d83547c93dbee7d581c,Чистая прибыль Сбербанка приблизилась к ₽1 трлн,Чистая прибыль Сбербанка приблизилась к ₽1 тр...,"сотрудников. в апреле—июне. человек, до 281,7 ..."
ccbb0b255ae458fe5eb523c334e8538e,Глава Центробанка Европы призвала регулировать...,Глава Центробанка Европы призвала регулироват...,При этом ведомство поддержало запрет на исполь...
2e6f9b06d46d2cdb1135ec5ca848feb7,Экс-премьер Франции Фийон вошел в совет директ...,Экс-премьер Франции Фийон вошел в совет дирек...,«Он не только управленец высочайшего уровня. В...
61d074a47c8a7d38d9ba48ca1c54b7d5,Marriott решила приостановить деятельность в Р...,Marriott решила приостановить деятельность в ...,РБК направил запрос в «Сафмар». Marriott прису...


## **2. Luhn Algorithm**

This is one of the earliest approaches of text summarization. Luhn proposed that the significance of each word in a document signifies how important it is. The idea is that any sentence with maximum occurances of the highest frequency words(Stopwords) and least occurances are not important to the meaning of document than others. Although it is not considered very accurate approach.

The Detailes of the algorithim can be found in the source link below.

source:
https://iq.opengenus.org/luhns-heuristic-method-for-text-summarization/


Resources:
- https://courses.ischool.berkeley.edu/i256/f06/papers/luhn58.pdf
- https://www.cs.cmu.edu/~nasmith/LS2/das-martins.07.pdf


#### **A. Perform Summarization on One Data Sample:**

In [64]:
original_text = doc['article_text'].iloc[0]
original_text

' Губернатор попросил не считать щебень полезным ископаемым Иначе налоги для его производителей вырастут в 15 раз Глава Карелии Артур Парфенчиков предложил вице-премьеру Андрею Белоусову изменить Налоговый кодекс, чтобы снизить нагрузку на производителей щебня. Им налоговики все чаще доначисляют НДПИ, что грозит ростом цен на стройматериалы Цены на необходимый в строительстве щебень могут вырасти, поскольку от производителей все чаще требуют заплатить налог на добычу полезных ископаемых (НДПИ), говорится в датированном 1 декабря письме главы Республики Карелия Артура Парфенчикова вице-премьеру Андрею Белоусову. Копия есть у РБК. Письмо поступило в аппарат правительства и будет рассмотрено в установленном порядке, подтвердили в пресс-службе правительства. РБК направил запрос представителям Белоусова, Парфенчикова и Федеральной налоговой службы. Ранее базой начисления НДПИ был строительный камень, из которого затем производится щебень, но в последнее время налоговики все чаще требуют пер

Unlike some algorithims, this algorithim needs stop words treatment

In [65]:
stopwords = nltk.corpus.stopwords.words('russian')
print(stopwords)

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

Now, we will define a function that does the required preprocessing for Luhn algorithim

In [66]:
def preprocess(text):
  formatted_text = text.lower()
  tokens = []
  for token in nltk.word_tokenize(formatted_text):
    tokens.append(token)
  tokens = [word for word in tokens if word not in stopwords and word not in string.punctuation]
  formatted_text = ' '.join(element for element in tokens)

  return formatted_text

In [67]:
formatted_text = preprocess(original_text)
formatted_text

'губернатор попросил считать щебень полезным ископаемым иначе налоги производителей вырастут 15 глава карелии артур парфенчиков предложил вице-премьеру андрею белоусову изменить налоговый кодекс снизить нагрузку производителей щебня налоговики чаще доначисляют ндпи грозит ростом цен стройматериалы цены необходимый строительстве щебень могут вырасти поскольку производителей чаще требуют заплатить налог добычу полезных ископаемых ндпи говорится датированном 1 декабря письме главы республики карелия артура парфенчикова вице-премьеру андрею белоусову копия рбк письмо поступило аппарат правительства рассмотрено установленном порядке подтвердили пресс-службе правительства рбк направил запрос представителям белоусова парфенчикова федеральной налоговой службы ранее базой начисления ндпи строительный камень которого затем производится щебень последнее время налоговики чаще требуют пересчета налога настаивая должен начисляться щебень предварительной оценке доначисление ндпи щебень увеличит налог

Below function will be used to calculate sentences score

In [68]:
def calculate_sentences_score(sentences, important_words, distance):
  scores = []
  sentence_index = 0

  for sentence in [nltk.word_tokenize(sentence) for sentence in sentences]:
    #print('------------')
    #print(sentence)

    word_index = []
    for word in important_words:
      #print(word)
      try:
        word_index.append(sentence.index(word))
      except ValueError:
        pass

    word_index.sort()
    #print(word_index)

    if len(word_index) == 0:
      continue

    # [0, 1, 5]
    groups_list = []
    group = [word_index[0]]
    i = 1 # 3
    while i < len(word_index): # 3
      # first execution: 1 - 0 = 1
      # second execution: 2 - 1 = 1
      if word_index[i] - word_index[i - 1] < distance:
        group.append(word_index[i])
        #print('group', group)
      else:
        groups_list.append(group[:])
        group = [word_index[i]]
        #print('group', group)
      i += 1
    groups_list.append(group)
    #print('all groups', groups_list)

    max_group_score = 0
    for g in groups_list:
      #print(g)
      important_words_in_group = len(g)
      total_words_in_group = g[-1] - g[0] + 1
      score = 1.0 * important_words_in_group**2 / total_words_in_group
      #print('group score', score)

      if score > max_group_score:
        max_group_score = score

    scores.append((max_group_score, sentence_index))
    sentence_index += 1

  #print('final scores', scores)
  return scores

In addition, we have developped a function that summarizes the text based on the scores

In [69]:
def summarize_Luhn(text, top_n_words, distance, number_of_sentences, percentage = 0):
    original_sentences = [sentence for sentence in nltk.sent_tokenize(text)]
    #print(original_sentences)
    formatted_sentences = [preprocess(original_sentence) for original_sentence in original_sentences]
    #print(formatted_sentences)
    words = [word for sentence in formatted_sentences for word in nltk.word_tokenize(sentence)]
    #print(words)
    frequency = nltk.FreqDist(words)
    #print(frequency)
    #return frequency
    top_n_words = [word[0] for word in frequency.most_common(top_n_words)]
    #print(top_n_words)
    sentences_score = calculate_sentences_score(formatted_sentences, top_n_words, distance)
    #print(sentences_score)
    if percentage > 0:
        best_sentences = heapq.nlargest(int(len(formatted_sentences) * percentage), sentences_score)
    else:  
        best_sentences = heapq.nlargest(number_of_sentences, sentences_score)
    #print(best_sentences)
    best_sentences = [original_sentences[i] for (score, i) in best_sentences]
    #print(best_sentences)
    return original_sentences, best_sentences, sentences_score

After defining the needed functions, it is the time for applying the summerization. 

In [70]:
original_sentences, best_sentences, sentences_score = summarize_Luhn(original_text, 5, 2, 4)

In [71]:
original_doc_sentences = original_sentences
original_sentences

[' Губернатор попросил не считать щебень полезным ископаемым Иначе налоги для его производителей вырастут в 15 раз Глава Карелии Артур Парфенчиков предложил вице-премьеру Андрею Белоусову изменить Налоговый кодекс, чтобы снизить нагрузку на производителей щебня.',
 'Им налоговики все чаще доначисляют НДПИ, что грозит ростом цен на стройматериалы Цены на необходимый в строительстве щебень могут вырасти, поскольку от производителей все чаще требуют заплатить налог на добычу полезных ископаемых (НДПИ), говорится в датированном 1 декабря письме главы Республики Карелия Артура Парфенчикова вице-премьеру Андрею Белоусову.',
 'Копия есть у РБК.',
 'Письмо поступило в аппарат правительства и будет рассмотрено в установленном порядке, подтвердили в пресс-службе правительства.',
 'РБК направил запрос представителям Белоусова, Парфенчикова и Федеральной налоговой службы.',
 'Ранее базой начисления НДПИ был строительный камень, из которого затем производится щебень, но в последнее время налоговики

In [72]:
best_sentences

['По словам вице-президента Национальной нерудной компании Сергея Глебова, щебень дороже строительного камня в пять-шесть раз.',
 'м щебня обошелся покупателям почти в 1,5 тыс.',
 'Парфенчиков просит Белоусова дать поручение «соответствующим федеральным исполнительным органам государственной власти» проработать вопрос об изменении налогового законодательства и уточнить порядок налогообложения при добыче полезных ископаемых.',
 'Письмо поступило в аппарат правительства и будет рассмотрено в установленном порядке, подтвердили в пресс-службе правительства.']

In [87]:
hilight_summary(original_sentences,best_sentences,sentences_format=1)

===============Summary=============================
  Губернатор попросил не считать щебень полезным ископаемым Иначе налоги для его производителей вырастут в 15 раз Глава Карелии Артур Парфенчиков
предложил вице-премьеру Андрею Белоусову изменить Налоговый кодекс, чтобы снизить нагрузку на производителей щебня. Им налоговики все чаще доначисляют
НДПИ, что грозит ростом цен на стройматериалы Цены на необходимый в строительстве щебень могут вырасти, поскольку от производителей все чаще требуют
заплатить налог на добычу полезных ископаемых (НДПИ), говорится в датированном 1 декабря письме главы Республики Карелия Артура Парфенчикова вице-
премьеру Андрею Белоусову. Копия есть у РБК. Письмо поступило в аппарат правительства и будет рассмотрено в установленном порядке, подтвердили в
пресс-службе правительства. РБК направил запрос представителям Белоусова, Парфенчикова и Федеральной налоговой службы. Ранее базой начисления НДПИ
был строительный камень, из которого затем производится щебень,

Now, we will apply the summarization to all the articles and store the summary in seperate column

In [88]:
Luhn_Summaries = []
for article in df['article_text']:
    original_sentences, best_sentences, sentences_score = summarize_Luhn(article, 5, 2, 4)
    summary_text = str('')
    for i in best_sentences:
        summary_text = summary_text + ("%s " % (i))
    Luhn_Summaries.append(summary_text)

df['Luhn_Summary'] = Luhn_Summaries

Drawing a randome article to see the summary

In [89]:
sample_article2 = df.sample()
hilight_summary(sample_article2['article_text'].iloc[0],sample_article2['Luhn_Summary'].iloc[0])

===============Summary=============================
  Ректор МГУ призвал оставить бакалавриат и магистратуру в отдельных сферах Это необходимо сделать в интересах иностранных студентов, считает ректор.
По его мнению, в стране стоит расширить число программ специалитета в особо значимых для общества и государства областях В России необходимо оставить
двухуровневую систему образования, которая сочетает в себе бакалавриат и магистратуру, для некоторых сфер. Об этом заявил академик РАН, ректор
Московского государственного университета имени М. В. Ломоносова Виктор Садовничий на заседании расширенного совета Российского союза ректоров,
передает «Интерфакс». По его словам, сделать это необходимо с учетом подготовки иностранных специалистов. При этом в новой системе образования должно
быть предусмотрено обучение по программам интегрированной магистратуры в наукоемких отраслях. Интегрированная магистратура подразумевает подготовку
специалистов, которые отучились по тому же направлению в бакала

## **3. Cosine similarity**
Cosine similarity is one of the metric to measure the text-similarity between two documents irrespective of their size in Natural language Processing. A word is represented into a vector form. The text documents are represented in n-dimensional vector space.

Mathematically, Cosine similarity metric measures the cosine of the angle between two n-dimensional vectors projected in a multi-dimensional space. The Cosine similarity of two documents will range from 0 to 1. If the Cosine similarity score is 1, it means two vectors have the same orientation. The value closer to 0 indicates that the two documents have less similarity.

Resources:
- https://en.wikipedia.org/wiki/Cosine_similarity 
- https://studymachinelearning.com/cosine-similarity-text-similarity-metric/


In [90]:
original_sentences = original_doc_sentences

We will continue using the same stopwords and preprocessing function we define earlier for Lhun. Also, we will keep using the same original text used previously.

Now, we will define a function to calculate sentence similarity using cosine similarity.

In [126]:
def calculate_sentence_similarity(sentence1, sentence2):
    words1 = [word for word in nltk.word_tokenize(sentence1)]
    words2 = [word for word in nltk.word_tokenize(sentence2)]
    #print(words1)
    #print(words2)

    all_words = list(set(words1 + words2))
    #print(all_words)

    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
    #print(vector1)
    #print(vector2)

    for word in words1: # Bag of words
    #print(word)
        vector1[all_words.index(word)] += 1
    for word in words2:
        vector2[all_words.index(word)] += 1

    #print(vector1)
    #print(vector2)

    return 1 - cosine_distance(vector1, vector2)

We are using the same original sentences and showing the similarity score for first and fourth setences

In [127]:
calculate_sentence_similarity(original_sentences[0], original_sentences[3])

0.1643989873053573

Below is a function that creates the similarity matrix:
- The higher the value, the greater the similarity between the sentences
- The more words in common, the greater the similarity

In [128]:
def calculate_similarity_matrix(sentences):
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
    #print(similarity_matrix)
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i == j:
                continue
            similarity_matrix[i][j] = calculate_sentence_similarity(sentences[i], sentences[j])
    return similarity_matrix

In [129]:
calculate_similarity_matrix(original_sentences)

array([[0.        , 0.28862169, 0.07352146, ..., 0.08108108, 0.07798129,
        0.16439899],
       [0.28862169, 0.        , 0.05234239, ..., 0.15393157, 0.14804664,
        0.11704115],
       [0.07352146, 0.05234239, 0.        , ..., 0.14704292, 0.07071068,
        0.        ],
       ...,
       [0.08108108, 0.15393157, 0.14704292, ..., 0.        , 0.10397505,
        0.        ],
       [0.07798129, 0.14804664, 0.07071068, ..., 0.10397505, 0.        ,
        0.        ],
       [0.16439899, 0.11704115, 0.        , ..., 0.        , 0.        ,
        0.        ]])

Since we sow what is the similarity matrix, we can use the same function to evaluate and get the best summary.

In [130]:
def summarize_cosine(text, number_of_sentences, percentage = 0):
    original_sentences = [sentence for sentence in nltk.sent_tokenize(text)]
    formatted_sentences = [preprocess(original_sentence) for original_sentence in original_sentences]
    similarity_matrix = calculate_similarity_matrix(formatted_sentences)

    similarity_graph = nx.from_numpy_array(similarity_matrix)


    scores = nx.pagerank_numpy(similarity_graph)
    ordered_scores = sorted(((scores[i], score) for i, score in enumerate(original_sentences)), reverse=True)

    if percentage > 0:
        number_of_sentences = int(len(formatted_sentences) * percentage)
        number_of_sentences = min(number_of_sentences, len(ordered_scores))
    best_sentences = []

    for sentence in range(number_of_sentences):
        best_sentences.append(ordered_scores[sentence][1])
  
    return original_sentences, best_sentences, ordered_scores

After developing all needed functions for evaluation and summarization, we need to perform the summary on the selected article. We selected the top four highly ranked sentences to for the summary.

In [131]:
original_sentences, best_sentences_cosine, scores = summarize_cosine(original_text, 4)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: networkx.pagerank_numpy is deprecated and will be removed in NetworkX 3.0, use networkx.pagerank instead.
  if __name__ == '__main__':


Now, let us take a look into the highly ranked sentences.

In [132]:
best_sentences_cosine

['Ранее базой начисления НДПИ был строительный камень, из которого затем производится щебень, но в последнее время налоговики все чаще требуют пересчета налога, настаивая, что он должен начисляться на щебень.',
 'Цены производителей на щебень в октябре 2021 года составляли, по информации Росстата, 586 руб.',
 'Им налоговики все чаще доначисляют НДПИ, что грозит ростом цен на стройматериалы Цены на необходимый в строительстве щебень могут вырасти, поскольку от производителей все чаще требуют заплатить налог на добычу полезных ископаемых (НДПИ), говорится в датированном 1 декабря письме главы Республики Карелия Артура Парфенчикова вице-премьеру Андрею Белоусову.',
 'По предварительной оценке, доначисление НДПИ на щебень увеличит налог в 3–15 раз, указывает Парфенчиков.']

Below are all the sentences along with their scours

It is the time to visualize the summary highlighted on the original context

In [133]:
hilight_summary(original_sentences,best_sentences_cosine,sentences_format=1)

===============Summary=============================
  Губернатор попросил не считать щебень полезным ископаемым Иначе налоги для его производителей вырастут в 15 раз Глава Карелии Артур Парфенчиков
предложил вице-премьеру Андрею Белоусову изменить Налоговый кодекс, чтобы снизить нагрузку на производителей щебня. Им налоговики все чаще
доначисляют НДПИ, что грозит ростом цен на стройматериалы Цены на необходимый в строительстве щебень могут вырасти, поскольку от производителей все
чаще требуют заплатить налог на добычу полезных ископаемых (НДПИ), говорится в датированном 1 декабря письме главы Республики Карелия Артура
Парфенчикова вице-премьеру Андрею Белоусову. Копия есть у РБК. Письмо поступило в аппарат правительства и будет рассмотрено в установленном
порядке, подтвердили в пресс-службе правительства. РБК направил запрос представителям Белоусова, Парфенчикова и Федеральной налоговой службы.
Ранее базой начисления НДПИ был строительный камень, из которого затем производится щебень, 

## **4. Sumy**

__Sumy__ is a library to extract summaries from HTML pages or plain texts. By utilizing __Sumy__, we will be performing two summarization techniques:
* TextRankSummarizer
* LSA Summarizer


For more information about Sumy, you could visit the official library's documentation page.
- library's pypi page: https://pypi.org/project/sumy/



### **A. Sumy Installation** 

In [ ]:
!pip install sumy

In [135]:
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer

### **B. TextRankSummarizer**
Text Rank is a kind of graph-based ranking algorithm used for recommendation purposes. TextRank is used in various applications where text sentences are involved. It worked on the ranking of text sentences and recursively computed based on information available in the entire text.

TextRank builds the graph related to the text. In a graph, each sentence is considered as vertex and each vertex is linked to the other vertex. These vertices cast a vote for another vertex. The importance of each vertex is defined by the higher number of votes. This importance is Here goal is to rank the sentences and this rank.

<br/>

Resources:
- https://medium.com/@ondenyi.eric/extractive-text-summarization-techniques-with-sumy-3d3b127a0a32
- https://machinelearninggeek.com/text-summarization-using-python/

##### **B.1 Perform Summarization on One Data Sample:**
Apply textranksummarizer to our test paragraph

In [136]:
def text_rank_summarizer(text, number_of_sentences):
    summarizer = TextRankSummarizer()
    parser = PlaintextParser.from_string(
      text,
      Tokenizer("russian"))
    summary = summarizer(parser.document, sentences_count= number_of_sentences)

    summary_text = ''
    for s in summary:
        summary_text = summary_text +' '+ str(s)
  
    return summary_text.strip()

In [137]:
hilight_summary(doc['article_text'].iloc[0],text_rank_summarizer(doc['article_text'].iloc[0],5))

===============Summary=============================
  Губернатор попросил не считать щебень полезным ископаемым Иначе налоги для его производителей вырастут в 15 раз Глава Карелии Артур Парфенчиков
предложил вице-премьеру Андрею Белоусову изменить Налоговый кодекс, чтобы снизить нагрузку на производителей щебня. Им налоговики все чаще
доначисляют НДПИ, что грозит ростом цен на стройматериалы Цены на необходимый в строительстве щебень могут вырасти, поскольку от производителей все
чаще требуют заплатить налог на добычу полезных ископаемых (НДПИ), говорится в датированном 1 декабря письме главы Республики Карелия Артура
Парфенчикова вице-премьеру Андрею Белоусову. Копия есть у РБК. Письмо поступило в аппарат правительства и будет рассмотрено в установленном
порядке, подтвердили в пресс-службе правительства. РБК направил запрос представителям Белоусова, Парфенчикова и Федеральной налоговой службы.
Ранее базой начисления НДПИ был строительный камень, из которого затем производится щебень, 

Apply the textranksummarizer to all article

In [138]:
text_rank_summaries = []
for article in df['article_text']:
    summary_text = text_rank_summarizer(article,5)
    text_rank_summaries.append(summary_text)

df['Sumy_Text_Rank_Summarizer_Summary'] = text_rank_summaries

In [139]:
sumdem = df.sample()
# sumdem['Sumy_Text_Rank_Summarizer_Summary'].iloc[0]
hilight_summary(sumdem['article_text'].iloc[0],sumdem['Sumy_Text_Rank_Summarizer_Summary'].iloc[0])

===============Summary=============================
  Reuters узнал о «редкой встрече» между Россией и Германией из-за Украины Reuters: переговорщики от России и ФРГ встретятся лично, чтобы обсудить
Украину, в январе Встреча должна состояться в январе, в ней будут участвовать замглавы администрации президента Дмитрий Козак и советник канцлера
Германии Йенс Плетнер. Агентство поясняет, что очные встречи чиновников такого уровня проходят редко Заместитель главы администрации президента
России Дмитрий Козак и советник канцлера ФРГ по внешнеполитическим вопросам Йенс Плетнер решили провести очную встречу по Украине в январе, сообщает
Reuters со ссылкой на источник в правительстве ФРГ. По его данным, об этом они договорились во время продолжительного телефонного разговора,
который состоялся в четверг. Reuters называет такую встречу «редкой», поясняя, что телефонные звонки по украинскому вопросу между главами
государств проводились в последнее время часто, а вот очные встречи между высокопос

### **C. LSA Summarizer**
Latent Semantic Analysis is based on Singular value decomposition(SVD). It reduces the data into lower-dimensional space. It performs spatial decomposition and captures information in a singular vector and the magnitude of so singular vector will represent the importance.


##### **C.1 Perform Summarization on One Data Sample:**
Apply textranksummarizer to our test paragraph

In [140]:
def LSA_summarizer(text, number_of_sentences):
    summarizer = LsaSummarizer()
    parser = PlaintextParser.from_string(
      text,
      Tokenizer("russian"))
    summary = summarizer(parser.document, sentences_count= number_of_sentences)

    summary_text = ''
    for s in summary:
        summary_text = summary_text + ' '+ str(s)

    return summary_text.strip()

In [141]:
hilight_summary(doc['article_text'].iloc[0],LSA_summarizer(doc['article_text'].iloc[0],5))

===============Summary=============================
  Губернатор попросил не считать щебень полезным ископаемым Иначе налоги для его производителей вырастут в 15 раз Глава Карелии Артур Парфенчиков
предложил вице-премьеру Андрею Белоусову изменить Налоговый кодекс, чтобы снизить нагрузку на производителей щебня. Им налоговики все чаще доначисляют
НДПИ, что грозит ростом цен на стройматериалы Цены на необходимый в строительстве щебень могут вырасти, поскольку от производителей все чаще требуют
заплатить налог на добычу полезных ископаемых (НДПИ), говорится в датированном 1 декабря письме главы Республики Карелия Артура Парфенчикова вице-
премьеру Андрею Белоусову. Копия есть у РБК. Письмо поступило в аппарат правительства и будет рассмотрено в установленном порядке, подтвердили в
пресс-службе правительства. РБК направил запрос представителям Белоусова, Парфенчикова и Федеральной налоговой службы. Ранее базой начисления НДПИ был
строительный камень, из которого затем производится щебень,

##### **B.2 Building a function to Perform Summarization on all data samples:**

Apply the LSA Summarizer to all article

In [142]:
lsa_summaries = []
for article in df['article_text']:
    summary_text = LSA_summarizer(article,5)
    lsa_summaries.append(summary_text)

df['LSA_Summarizer_Summary'] = lsa_summaries

In [143]:
sumdem = df.sample()
# sumdem['Sumy_Text_Rank_Summarizer_Summary'].iloc[0]
hilight_summary(sumdem['article_text'].iloc[0],sumdem['LSA_Summarizer_Summary'].iloc[0])

===============Summary=============================
  Лавров заявил о согласовании автокоридора между Арменией и Азербайджаном Согласование маршрута приведет к разблокировке автомобильного сообщения в
регионе, которое было прервано 30 лет назад, во время первой войны в Нагорном Карабахе Завершается согласование автомобильного транспортного коридора
между Арменией и Азербайджаном, сообщил министр иностранных дел России Сергей Лавров, который находится с визитом в Ереване. «Как известно,
железнодорожный маршрут согласован, сейчас завершается согласование автомобильного маршрута и согласование режима [функционирования транспортного
коридора], который будет установлен», — отметил глава российского МИДа (цитата по ТАСС). По словам Лаврова, режим проезда будет упрощенным, но с
опорой «на признание суверенитета армянской территории». Он подчеркнул, что речь идет о создании инфраструктуры «на долгие годы», так что в этом
вопросе «лучше все десять раз взвесить, прежде чем рубить с плеча». Минис

## **5. Pysummarization**

__pysummarization__ is Python library for the automatic summarization, document abstraction, and text filtering.

- https://pypi.org/project/pysummarization/

### **A. Pysummarization Installation**


In [ ]:
!pip install pysummarization

In [118]:
from pysummarization.nlpbase.auto_abstractor import AutoAbstractor
from pysummarization.tokenizabledoc.simple_tokenizer import SimpleTokenizer
from pysummarization.abstractabledoc.top_n_rank_abstractor import TopNRankAbstractor

### **B. Perform Summarization on One Data Sample:**
Apply textranksummarizer to our test paragraph

In [112]:
auto_abstractor = AutoAbstractor()
auto_abstractor.tokenizable_doc = SimpleTokenizer()
auto_abstractor.delimiter_list = [".", "\n"]
abstractable_doc = TopNRankAbstractor()

In [113]:
summary = auto_abstractor.summarize(doc['article_text'].iloc[0], abstractable_doc)

In [ ]:
summary

applying it to our document 

In [115]:
best_sentences = ''
for sentence in summary['summarize_result']:
    best_sentences = best_sentences  + sentence

In [116]:
hilight_summary(doc['article_text'].iloc[0],best_sentences)

===============Summary=============================
  Губернатор попросил не считать щебень полезным ископаемым Иначе налоги для его производителей вырастут в 15 раз Глава Карелии Артур Парфенчиков
предложил вице-премьеру Андрею Белоусову изменить Налоговый кодекс, чтобы снизить нагрузку на производителей щебня. Им налоговики все чаще
доначисляют НДПИ, что грозит ростом цен на стройматериалы Цены на необходимый в строительстве щебень могут вырасти, поскольку от производителей все
чаще требуют заплатить налог на добычу полезных ископаемых (НДПИ), говорится в датированном 1 декабря письме главы Республики Карелия Артура
Парфенчикова вице-премьеру Андрею Белоусову. Копия есть у РБК. Письмо поступило в аппарат правительства и будет рассмотрено в
установленном порядке, подтвердили в пресс-службе правительства. РБК направил запрос представителям Белоусова, Парфенчикова и Федеральной
налоговой службы. Ранее базой начисления НДПИ был строительный камень, из которого затем производится щебень, 

### **C. Building a function to Perform Summarization on all data samples:**

In [101]:
pysummarization_list = []
for article in df['article_text']:
    summary = auto_abstractor.summarize(article, abstractable_doc)
    best_sentences = ''
    for sentence in summary['summarize_result']:
        best_sentences = best_sentences  + sentence

    pysummarization_list.append(best_sentences.strip())

df['pysummarization_summary'] = pysummarization_list

In [102]:
sumdem = df.sample()
# sumdem['Sumy_Text_Rank_Summarizer_Summary'].iloc[0]
hilight_summary(sumdem['article_text'].iloc[0],sumdem['pysummarization_summary'].iloc[0])

===============Summary=============================
  Bloomberg узнал о закрытии «Ренессанс Капитала» в Лондоне и Нью-Йорке Инвестбанк Михаила Прохорова закроет отделения в Великобритании и США,
сообщили источники Bloomberg. Прохорову до сих пор удалось избежать попадания под санкции ЕС и США. В компании РБК заявили, что речь
идет о реструктуризации Российский инвестиционный банк «Ренессанс Капитал» (RenCap) закроет офис и исследовательский отдел в Нью-Йорке, сообщает
Bloomberg со ссылкой на источники. По словам собеседников агентства, которые просили не называть их имен, инвестбанк также может закрыть свой
офис в Лондоне. «Речь идет о реструктуризации наших компаний Renaissance Capital Limited и RenCap Securities Inc в Лондоне и Нью-Йорке.
Это именно реструктуризация, не закрытие», — сообщил РБК представитель «Ренессанс Капитала». www.adv.rbc.ru «Ренессанс Капитал» — один из
старейших инвестиционных банков России, сейчас он на 100% принадлежит группе ОНЭКСИМ Михаила Прохорова и специа

In [103]:
sample_article3 = df.sample()
print("TFIDF")
hilight_summary(sample_article3['article_text'].iloc[0],sample_article3['TF_IDF_Summary'].iloc[0])
print("=========================")
print("Luhn")
hilight_summary(sample_article3['article_text'].iloc[0],sample_article3['Luhn_Summary'].iloc[0])
print("pysummarization_summary")
hilight_summary(sample_article3['article_text'].iloc[0],sample_article3['pysummarization_summary'].iloc[0])
print("=========================")


TFIDF
===============Summary=============================
  ЦБ предложил банкам возвращать украденные мошенниками деньги за 30 дней В каких случаях они должны будут это делать ЦБ готовит изменения в закон для
улучшения ситуации с возвратом украденных у граждан средств. Поправки, в частности, ограничивают срок такого возврата для банков 30 днями. В 2021 году
банки вернули меньше 7% похищенных денег Банк России планирует установить для кредитных организаций обязательный срок возврата украденных денежных
средств. При установлении факта денежного перевода без согласия клиента банк должен будет вернуть ему деньги в течение 30 дней после получения
соответствующего заявления, следует из поправок в 161-ФЗ «О национальной платежной системе», с которыми ознакомился РБК. Подлинность документа
подтвердил источник на платежном рынке. Планы по введению обязательного срока для возврата подтвердил представитель ЦБ. «Действующим законом не
предусмотрены сроки, в течение которых банки должны возмещать к

## **6- Bidirectional Encoder Representations from Transformers (BERT)**
As defined in the Wikipedia, __BERT__ is a transformer-based machine learning technique for natural language processing (NLP) pre-training developed by Google.
Although, __BERT__ is not considered as part of the classical language models for text summarization. However, we will include a lighter implementation of it as a transition between this notebook and the advanced ones.

- https://pypi.org/project/bert-extractive-summarizer/

### **A. BERT Installation**

In [ ]:
!pip install bert-extractive-summarizer

In [ ]:
!pip install git+https://github.com/huggingface/transformers

In [ ]:
!pip install sacremoses

### **B. Summarization on One Data Sample:**

In [121]:
from transformers import BigBirdTokenizer
from summarizer import Summarizer

In [ ]:
summarizer = Summarizer()

In [123]:
summary = summarizer(doc['article_text'].iloc[0])

In [124]:
hilight_summary(doc['article_text'].iloc[0],summary)

===============Summary=============================
  Губернатор попросил не считать щебень полезным ископаемым Иначе налоги для его производителей вырастут в 15 раз Глава Карелии Артур Парфенчиков
предложил вице-премьеру Андрею Белоусову изменить Налоговый кодекс, чтобы снизить нагрузку на производителей щебня. Им налоговики все чаще доначисляют
НДПИ, что грозит ростом цен на стройматериалы Цены на необходимый в строительстве щебень могут вырасти, поскольку от производителей все чаще требуют
заплатить налог на добычу полезных ископаемых (НДПИ), говорится в датированном 1 декабря письме главы Республики Карелия Артура Парфенчикова вице-
премьеру Андрею Белоусову. Копия есть у РБК. Письмо поступило в аппарат правительства и будет рассмотрено в установленном порядке, подтвердили в
пресс-службе правительства. РБК направил запрос представителям Белоусова, Парфенчикова и Федеральной налоговой службы. Ранее базой начисления
НДПИ был строительный камень, из которого затем производится щебень,

In [5]:
df = pd.read_pickle('../outputs/pickles/extractive_summarization.pkl')

In [31]:
print("="*50)
print('\n')
print("TFIDF")
hilight_summary(sample_article3['body'].iloc[0],sample_article3['TF_IDF_Summary'].iloc[0])
print("="*50)
print('\n')
print("="*50)
print("Luhn")
hilight_summary(sample_article3['body'].iloc[0],sample_article3['Luhn_Summary'].iloc[0])
print('\n')
print("="*50)
print("pysummarization_summary")
hilight_summary(sample_article3['body'].iloc[0],sample_article3['pysummarization_summary'].iloc[0])
print("="*50)



TFIDF
===============Summary=============================
  Путин отменил требование к экспортерам продавать 50% валютной выручки Теперь долю обязательной продажи валютной выручки будет устанавливать
правительственная комиссия Доллар/Рубль USD/RUB ₽59,350 -0,08% Купить Евро/Рубль EUR/RUB ₽63,300 -0,46% Купить Президент Владимир Путин отменил
требование об обязательной продаже 50% экспортной выручки в иностранной валюте. Указ опубликован на портале правовой информации. Само
требование для экспортеров по продаже выручки осталось, но ее размер теперь будет определять правкомиссия по контролю за осуществлением иностранных
инвестиций. Срок установит совет директоров ЦБ. www.adv.rbc.ru С 28 февраля экспортеров обязали продавать 80% валютной выручки в рамках всех
внешнеторговых договоров в течение трех рабочих дней со дня ее поступления. Мера должна была в том числе поддержать рубль. Она начала
действовать вскоре после введения западных санкций и заморозки золотовалютных резервов за рубежом

In [14]:
sample_article3 = df.sample()

In [35]:
print(list(sample_article3['TF_IDF_Summary']))

['Срок установит совет директоров ЦБ. Указ опубликован на портале правовой информации. Мера должна была в том числе поддержать рубль. Регулятор также разрешил им продавать валюту на счета своих уполномоченных банков. В конце мая также была снижена с 80 до 50% доля обязательной продажи валютной выручки экспортерами. ']


In [37]:
print(list(sample_article3['Luhn_Summary']))

[' Путин отменил требование к экспортерам продавать 50% валютной выручки Теперь долю обязательной продажи валютной выручки будет устанавливать правительственная комиссия Доллар/Рубль USD/RUB ₽59,350 -0,08% Купить Евро/Рубль EUR/RUB ₽63,300 -0,46% Купить Президент Владимир Путин отменил требование об обязательной продаже 50% экспортной выручки в иностранной валюте. Она начала действовать вскоре после введения западных санкций и заморозки золотовалютных резервов за рубежом, последовавших в ответ на российскую военную спецоперацию на Украине. Мера должна была в том числе поддержать рубль. Само требование для экспортеров по продаже выручки осталось, но ее размер теперь будет определять правкомиссия по контролю за осуществлением иностранных инвестиций. ']


In [38]:
print(list(sample_article3['pysummarization_summary']))

['Путин отменил требование к экспортерам продавать 50% валютной выручки Теперь долю обязательной продажи валютной выручки будет устанавливать правительственная комиссия Доллар/Рубль USD/RUB ₽59,350 -0,08% Купить Евро/Рубль EUR/RUB ₽63,300 -0,46% Купить Президент Владимир Путин отменил требование об обязательной продаже 50% экспортной выручки в иностранной валюте.\n Указ опубликован на портале правовой информации.\n Само требование для экспортеров по продаже выручки осталось, но ее размер теперь будет определять правкомиссия по контролю за осуществлением иностранных инвестиций.\n Срок установит совет директоров ЦБ.\nru С 28 февраля экспортеров обязали продавать 80% валютной выручки в рамках всех внешнеторговых договоров в течение трех рабочих дней со дня ее поступления.\n Мера должна была в том числе поддержать рубль.\n Она начала действовать вскоре после введения западных санкций и заморозки золотовалютных резервов за рубежом, последовавших в ответ на российскую военную спецоперацию на

In [41]:
print(list(sample_article3['LSA_Summarizer_Summary']))

['www.adv.rbc.ru В апреле срок продажи выручки для экспортеров несырьевого неэнергетического сектора увеличили с трех до 60 рабочих дней. Регулятор также разрешил им продавать валюту на счета своих уполномоченных банков. Затем время, за которое экспортеры должны продать часть своей валютной выручки, выросло до 120 рабочих дней. В конце мая также была снижена с 80 до 50% доля обязательной продажи валютной выручки экспортерами. Свое решение Минфин объяснил стабилизацией курса рубля и «достижением достаточного уровня ликвидности в иностранной валюте на внутреннем валютном рынке».']


In [ ]:
print(list(sample_article3['LSA_Summarizer_Summary']))

In [39]:
sample_article3

,overview,headline,body,category,tags,TF_IDF_Summary,Luhn_Summary,Sumy_Text_Rank_Summarizer_Summary,LSA_Summarizer_Summary,pysummarization_summary
8a05a4288503baf97b9590149da0f22a,Теперь долю обязательной продажи валютной выру...,Путин отменил требование к экспортерам продава...,Путин отменил требование к экспортерам продав...,economics,"[Владимир Путин, валютная выручка, экспортеры]",Срок установит совет директоров ЦБ. Указ опубл...,Путин отменил требование к экспортерам продав...,Путин отменил требование к экспортерам продава...,www.adv.rbc.ru В апреле срок продажи выручки д...,Путин отменил требование к экспортерам продава...
